## Rating csv


In [1]:
import pandas as pd
import datetime
import numpy as  np
from tqdm import tqdm
np.random.seed(1337)
with open('./kaggle/rating_train.csv', 'r') as f:
    ls = f.readlines()[1:]
u_map = {}

dates = []
foods = []
users = []
    

with tqdm(total=len(ls)) as pbar:
    for l in ls:
        date_str, user, food = l.strip().split(',')
        date = datetime.datetime.strptime(date_str, '%Y-%m-%d')
        user, food = int(user), int(food)
        if user not in u_map:
            u_map[user] = []
        u_map[user].append( (date, food) )
        
        dates.append(date)
        foods.append(food)
        users.append(user)
        pbar.update(1)
        

        
food_map = {f:i for i, f in enumerate(set(foods))}
user_map = {u:i for i, u in enumerate(set(users))}



100%|██████████| 2681494/2681494 [00:19<00:00, 134962.82it/s]


### Generate training data and label

In [2]:
### ALS embeddingimport numpy as  np
from scipy.sparse import csr_matrix
from scipy.spatial.distance import cosine
with tqdm(total=len(u_map)) as pbar:
    
#     split 7 days for future
    eaten = np.zeros([len(user_map), len(food_map)])
    pairs = []
    for user,v in u_map.items():
        history = v
        ds = np.array([d for d,f in history])
        fs = np.array([f for d,f in history])
        date_list = sorted(set(ds))
        rest_date_set = date_list[-7:]

        for food, date in zip(fs,ds):
            eaten[user_map[user], food_map[food]] = 1
            if date not in rest_date_set:
                pair = (user_map[user], food_map[f])
                pairs.append(pair)
        pbar.update(1)
    rows = []
    cols = []
    pairs = set(pairs)
    for pair in pairs:
        u, f = pair
        rows.append(u)
        cols.append(f)
    data = np.ones([len(rows),])
    R = csr_matrix((data, (rows,cols)), shape=(len(user_map),len(food_map)))
print R.shape
print 'Done'


100%|██████████| 2608/2608 [00:04<00:00, 594.04it/s]

(2608, 5532)
Done


### ALS embedding Training

In [8]:
import implicit
import prettytable
from implicit.als import AlternatingLeastSquares
from implicit.approximate_als import (AnnoyAlternatingLeastSquares, FaissAlternatingLeastSquares,
                                      NMSLibAlternatingLeastSquares)
from implicit.bpr import BayesianPersonalizedRanking
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight)
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
# initialize a model
MODELS = {"als":  AlternatingLeastSquares,
#           "nmslib_als": NMSLibAlternatingLeastSquares,
          "annoy_als": AnnoyAlternatingLeastSquares,
#           "faiss_als": FaissAlternatingLeastSquares,
#           "tfidf": TFIDFRecommender,
#           "cosine": CosineRecommender,
          "bpr": BayesianPersonalizedRanking,
#           "bm25": BM25Recommender
         }
user_items = R.transpose()
    
rev = {v:k for k,v in user_map.items()}
F = [32*2**(i)for i in range(5)]
iters = [10+100*(i) for i in range(3)]

params = {'use_gpu':True}
T = prettytable.PrettyTable()
T.field_names = ["model name", "factor", "iter", "hit rate"]
best = 0.
for s in MODELS:
    for factor in F:
        for it in iters:
            model_class = MODELS.get(s)
            if s == "bpr":
                params = {'use_gpu':True, 'verify_negative_samples':True}
            else:
                params = {'use_gpu':True}

            params['factors'] = factor
            params['iterations'] = it
            
            model = model_class(**params)
            model.fit(user_items, show_progress=False)
            hits = []
            kkk=20
            idxs = np.random.permutation(len(user_map))
            for idx in idxs[:500]:
                recommendations = model.recommend(idx, user_items, N=kkk)
                hit = [eaten[idx,i] != 0 for i,_ in recommendations]
                hits.append(np.mean(hit))
                # hit = [arr[i]==1 and Y_map[rev[i]][0,i] == 1for i,_ in recommendations]
            T.add_row([s, factor, it, np.mean(hits)])
            if np.mean(hits) >  best:
                best = np.mean(hits)
                attr = [s, factor, it, np.mean(hits)]
                best_model = model
print attr
print T

['als', 256, 110, 1.0]
+------------+--------+------+--------------------+
| model name | factor | iter |      hit rate      |
+------------+--------+------+--------------------+
|    bpr     |   32   |  10  | 0.8009000000000001 |
|    bpr     |   32   | 110  | 0.8225999999999999 |
|    bpr     |   32   | 210  |       0.7048       |
|    bpr     |   64   |  10  |       0.8038       |
|    bpr     |   64   | 110  |       0.8404       |
|    bpr     |   64   | 210  |       0.7952       |
|    bpr     |  128   |  10  |       0.8025       |
|    bpr     |  128   | 110  |       0.8627       |
|    bpr     |  128   | 210  |       0.8957       |
|    bpr     |  256   |  10  |       0.7955       |
|    bpr     |  256   | 110  | 0.8627999999999999 |
|    bpr     |  256   | 210  |       0.9322       |
|    bpr     |  512   |  10  |       0.8035       |
|    bpr     |  512   | 110  |       0.8665       |
|    bpr     |  512   | 210  |       0.9439       |
|    als     |   32   |  10  | 0.92389999

### ALT testing with best validation model


In [9]:
### ALS embeddingimport numpy as  np
from scipy.sparse import csr_matrix
from scipy.spatial.distance import cosine
with tqdm(total=len(u_map)) as pbar:
    
#     split 1/10 for future
    pairs = []
    for user,v in u_map.items():
        history = v
#         calculate the Y
        for d,f in history:
            pair = (user_map[user], food_map[f])
            pairs.append(pair)
        pbar.update(1)
    rows = []
    cols = []
    for pair in pairs:
        u, f = pair
        rows.append(u)
        cols.append(f)
    data = np.ones([len(rows),])
    RR = csr_matrix((data, (rows,cols)), shape=(len(user_map),len(food_map)))
    rows = []
    cols = []
    pairs = set(pairs)
    for pair in pairs:
        u, f = pair
        rows.append(u)
        cols.append(f)
    data = np.ones([len(rows),])
    R = csr_matrix((data, (rows,cols)), shape=(len(user_map),len(food_map)))
print R.shape
print 'Done'
user_items = R.transpose()
model = best_model
model.fit(user_items, show_progress=True)
            
rev_food_map = {v:k for k,v in food_map.items()}
with tqdm(total=len(u_map)) as pbar:
    with open('./ALS_predict.csv', 'w') as f:
        f.write('userid,foodid\n')
        for user in user_map.keys():
#             most_eat = np.argmax(RR[user_map[user],:])
#             most_food_emb = model.item_factors[most_eat,:]
#             distance = np.matmul(model.item_factors, np.expand_dims(most_food_emb,axis=-1)).flatten()
#             norms = np.linalg.norm(model.item_factors, axis=-1)
#             distance = -distance/ norms
            
#             recommendations = [i for i in np.argsort(distance)]
            recommendations = [i for i,_ in model.recommend(user_map[user], user_items, N=len(food_map))]
            s = ''
            c = 0
            for food_idx in recommendations:
                if c == 20:
                    break
                if R[user_map[user], food_idx] == 0:
                    s += ' %d' % rev_food_map[food_idx]
                    c+=1
            f.write('%d,%s\n' % (user,s) )
            pbar.update(1)
print 'Done'
# for u in range()


100%|██████████| 2608/2608 [00:01<00:00, 1561.79it/s]
  3%|▎         | 3.5/110 [00:00<00:03, 34.67it/s]

(2608, 5532)
Done


100%|██████████| 110.0/110 [00:01<00:00, 58.48it/s]
100%|██████████| 2608/2608 [00:34<00:00, 76.62it/s]

Done
